# <p style = "color : #f9564f"><strong>Data Mining Cup 2022 - Mannheim University - Team Runtime Terror</strong>
This semester task is about the prediction of repleshiment of an online (e-commerce) shop. Therefor we do have continous data for a time range of approx 7 Months in total (01.06.2020 - 31.01.2021) as training data<br>
The prediction is to be fulilled on the test data. The test data has a start from the 01.12.2020 till the 28.12.2020.
- Sources of data is provided in <a href="https://ilias.uni-mannheim.de/ilias.php?ref_id=1270781&cmd=view&cmdClass=ilrepositorygui&cmdNode=xk&baseClass=ilRepositoryGUI"><strong>ILIAS</strong></a>
- Submission and milestone upload on <a href="http://ilias2.informatik.uni-mannheim.de/ilias.php?ref_id=77&cmd=infoScreen&cmdClass=ilobjtestgui&cmdNode=u1:oq&baseClass=ilrepositorygui"><strong>ILIAS2</strong></a>
    - For <u>ILIAS2</u> the <a href="https://www.uni-mannheim.de/it/anleitungen/vpn/"><strong>VPN</strong></a> offered by the University´s IT-Department is necessary<br></br>

<p style = "color : #fd9e02"><strong>Loading necessary libraries for this project ---May appended by more libraries during the proceeding of this project---- All libraries are collected here</strong></p>

In [1]:
# Operating System - Settings - Warnings - Datetime - Formatting
import os.path, sys, warnings

def fxn():
    warnings.warn('deprecated', DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    fxn()

from datetime import datetime
dateparse = lambda x: pd.to_datetime(x, format='%Y-%m-%d') if x.startswith('20') else np.nan
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

# Start - Loading Libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import dash
import plotly.express as px

# Simple Preprocessing - Encoding - Normalization - Discretizaion
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Model Implementation

In [2]:
# Loading Data into separate dataframes
df_items = pd.read_csv('../../Data/items.csv', delimiter="|", index_col=False)
df_orders = pd.read_csv("../../Data/orders.csv", delimiter="|", index_col=False, date_parser=dateparse, parse_dates = ['date'])
df_cat_hierarchy = pd.read_csv("../../Data/category_hierarchy.csv", delimiter="|", index_col = False)

# Saving typing time
def first_insights(df):
    print(color.BOLD + 'First insights' + color.END)
    display(df.head())
    print('')
    print(color.BOLD + 'Information about dataset' + color.END)
    display(df.info())
    display(df.describe(include = 'all'))
    print('Shape of dataset: ', df.shape)
    print('Index of dataset: ', df.index)

In [3]:
# Looking at items dataset
first_insights(df_items)

First insights


,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories
0,22665,861,4,0,490,2,66,"[2890, 855, 3908, 3909]"
1,28640,1366,10,1,537,0,101,NaN
2,13526,1090,10,0,511,0,0,"[3270, 163, 284, 1694, 12, 3837, 2422, 3595, 3..."
3,21399,1090,10,1,511,0,0,[3270]
4,8504,768,4,1,484,0,66,[2470]



Information about dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32776 entries, 0 to 32775
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   itemID      32776 non-null  int64 
 1   brand       32776 non-null  int64 
 2   feature_1   32776 non-null  int64 
 3   feature_2   32776 non-null  int64 
 4   feature_3   32776 non-null  int64 
 5   feature_4   32776 non-null  int64 
 6   feature_5   32776 non-null  int64 
 7   categories  25988 non-null  object
dtypes: int64(7), object(1)
memory usage: 2.0+ MB


None

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories
count,32776.000000,32776.000000,32776.000000,32776.000000,32776.000000,32776.000000,32776.000000,25988
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12191
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1807]
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,346
mean,16387.500000,702.776574,6.451062,0.817946,440.280388,0.804735,74.689285,NaN
std,9461.760548,432.774074,2.744884,0.778284,132.566994,1.354669,57.621143,NaN
min,0.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,NaN
25%,8193.750000,322.000000,4.000000,0.000000,453.000000,0.000000,17.000000,NaN
50%,16387.500000,648.000000,6.000000,1.000000,491.000000,0.000000,66.000000,NaN
75%,24581.250000,1073.000000,10.000000,1.000000,509.000000,3.000000,126.000000,NaN


Shape of dataset:  (32776, 8)
Index of dataset:  RangeIndex(start=0, stop=32776, step=1)


In [4]:
# Looking at order dataset
first_insights(df_orders)

First insights


,date,userID,itemID,order
0,2020-06-01,38769,3477,1
1,2020-06-01,42535,30474,1
2,2020-06-01,42535,15833,1
3,2020-06-01,42535,20131,1
4,2020-06-01,42535,4325,1



Information about dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071538 entries, 0 to 1071537
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   date    1071538 non-null  datetime64[ns]
 1   userID  1071538 non-null  int64         
 2   itemID  1071538 non-null  int64         
 3   order   1071538 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 32.7 MB


None

/var/folders/q8/lvnrt_3s0_b_tzxcnkv3y1340000gn/T/ipykernel_2913/157844962.py:13: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  display(df.describe(include = 'all'))


,date,userID,itemID,order
count,1071538,1.071538e+06,1.071538e+06,1.071538e+06
unique,245,NaN,NaN,NaN
top,2021-01-30 00:00:00,NaN,NaN,NaN
freq,8109,NaN,NaN,NaN
first,2020-06-01 00:00:00,NaN,NaN,NaN
last,2021-01-31 00:00:00,NaN,NaN,NaN
mean,NaN,2.307968e+04,1.652196e+04,1.399758e+00
std,NaN,1.331930e+04,9.451299e+03,1.479233e+00
min,NaN,0.000000e+00,0.000000e+00,1.000000e+00
25%,NaN,1.152400e+04,8.350000e+03,1.000000e+00


Shape of dataset:  (1071538, 4)
Index of dataset:  RangeIndex(start=0, stop=1071538, step=1)


In [5]:
# Looking at category hierarchy dataset
first_insights(df_cat_hierarchy)

First insights


,category,parent_category
0,0,75
1,1,1499
2,2,1082
3,3,3498
4,4,1623



Information about dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332 entries, 0 to 4331
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   category         4332 non-null   int64
 1   parent_category  4332 non-null   int64
dtypes: int64(2)
memory usage: 67.8 KB


None

,category,parent_category
count,4332.000000,4332.000000
mean,2147.165512,2204.845568
std,1241.039814,1222.376330
min,0.000000,8.000000
25%,1069.750000,1143.750000
50%,2146.500000,2284.000000
75%,3221.250000,3329.000000
max,4299.000000,3898.000000


Shape of dataset:  (4332, 2)
Index of dataset:  RangeIndex(start=0, stop=4332, step=1)


In [6]:
pd_merged = pd.merge(df_orders, df_items, on = 'itemID', how = "left")
display(pd_merged.head())

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories
0,2020-06-01,38769,3477,1,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]"
1,2020-06-01,42535,30474,1,193,10,3,229,3,132,"[3459, 3738, 679, 1628, 4072]"
2,2020-06-01,42535,15833,1,1318,4,1,455,0,108,"[2973, 2907, 2749, 3357]"
3,2020-06-01,42535,20131,1,347,4,0,291,3,44,"[30, 1515, 1760, 2932, 1287, 2615, 3727, 2450,..."
4,2020-06-01,42535,4325,1,539,6,0,303,0,45,"[3104, 1772, 2029, 1274, 3915, 888, 1118, 3882..."
